In [1]:
import warnings

import ipywidgets as widgets
import kmapper as km
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hdbscan import HDBSCAN
from IPython.display import HTML, clear_output, display
from ipywidgets import fixed, interact, interact_manual, interactive
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import IsolationForest
from umap import UMAP

import graph_tool.all as gt

warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline


## Data Cleaning

We're using the a Big Five Personality Factors dataset from [here](https://openpsychometrics.org/_rawdata/), with the idea that there should be a grouping of *five* that we might be able to pull out from the data.

The only real data cleaning step is to reverse some of the negatively phrased questions for each factor (i.e. a higher score should align with a higher factor score). This will just make interpreting and analysis easier. [See here](https://ipip.ori.org/new_ipip-50-item-scale.htm) for more info on the questions.

In [2]:
df = pd.read_csv('../data/data.csv', sep="\t")

In [3]:
df.head()

,race,age,engnat,gender,hand,source,country,E1,E2,E3,...,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
0,3,53,1,1,1,1,US,4,2,5,...,4,1,3,1,5,1,4,2,5,5
1,13,46,1,2,1,1,US,2,2,3,...,3,3,3,3,2,3,3,1,3,2
2,1,14,2,2,1,1,PK,5,1,1,...,4,5,5,1,5,1,5,5,5,5
3,3,19,2,2,1,1,RO,2,5,2,...,4,3,5,2,4,2,5,2,5,5
4,11,25,2,2,1,2,US,3,1,3,...,3,1,1,1,3,1,3,1,5,3


In [4]:
demo_cols = ['race', 'age', 'engnat', 'gender', 'hand', 'source', 'country']
question_columns = [col for col in df.columns if col not in demo_cols]
o_columns = [col for col in df.columns if col.startswith("O")]
c_columns = [col for col in df.columns if col.startswith("C")]
e_columns = [col for col in df.columns if col.startswith("E")]
a_columns = [col for col in df.columns if col.startswith("A")]
n_columns = [col for col in df.columns if col.startswith("N")]

In [5]:
flip_qs = 'E2 E4 E6 E8 E10 N2 N4 A1 A3 A5 A7 C2 C4 C6 C8 O2 O4 O6'.split()
for q in flip_qs:
    df[q] = df[q].apply(lambda x: abs(x-6))

In [6]:
for columns in [o_columns, c_columns, e_columns, a_columns, n_columns]:
    assert (df[columns].corr() > 0).all(axis=None)

In [7]:
X = df[question_columns].drop_duplicates()

## Creating the Projection Interface
We'll need 4 parameters for MAPPER: the lens, the clusterer, the number of cubes, and the overlap percentage. Since both the lens creation and the mapper can be computationally intensive, we'll include a run button to run on command rather than update continually.

### Create Helper Functions

In [8]:
def get_lens(value):
    if value == "svd":
        mapper = km.KeplerMapper(verbose=0)
        projection = TruncatedSVD(n_components=2, random_state=1234)
        lens = mapper.fit_transform(X, projection=projection)
    if value == "iso_l2":
        model = IsolationForest(random_state=1234, contamination="auto", behaviour="new")
        model.fit(X)
        lens1 = model.decision_function(X).reshape((X.shape[0], 1))
        
        mapper = km.KeplerMapper(verbose=0)
        lens2 = mapper.fit_transform(X, projection="l2norm")

        lens = np.c_[lens1, lens2]
    if value == "umap":
        mapper = km.KeplerMapper(verbose=0)
        projection = UMAP(n_neighbors=30, min_dist=0.01, random_state=1234)
        lens = mapper.fit_transform(
            X, projection=projection)
    return lens

def get_clusterer(value):
    if value == "dbscan":
        clusterer = DBSCAN(eps=0.5, min_samples=5)
    if value == "hdbscan":
        clusterer = HDBSCAN(allow_single_cluster=True)
    if value == "agglomerative_3":
        clusterer = AgglomerativeClustering(n_clusters=3)
    if value == "kmeans_3":
        clusterer = KMeans(n_clusters=3)
    return clusterer

### Define Buttons 

In [9]:
# Input Widgets

lens_selector = widgets.Dropdown(
    options=['svd', 'iso_l2', 'umap'],
    value="svd",
    description="Projection")

clusterer_selector = widgets.Dropdown(
    options=['dbscan', 'hdbscan', 'agglomerative_3', 'kmeans_3'],
    value="dbscan",
    description="Clusterer")

cubes_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description="N Cubes",
    disabled=False,
    continuous_update=False,
)

overlap_slider = widgets.FloatSlider(
    value=0.5,
    min=0.05,
    max=1.0,
    step=0.05,
    description="Overlap Percentage",
    disabled=False,
    continuous_update=False,
    style={"description_width": "initial"},
)

run_button = widgets.Button(description="Run", style={"width": "100%"})

# Output Widgets
out_image = widgets.Output()

# Layout
controls = widgets.VBox(
    [lens_selector, clusterer_selector, cubes_slider, overlap_slider, run_button]
)

layout = widgets.HBox([controls, out_image])

In [10]:
def mapper_to_graph(mapper_graph):
    _node_map = {node : index for index, node in enumerate(sorted(mapper_graph['nodes'].keys()))}
    
    _g = gt.Graph(directed=False)
    _g.add_vertex(len(_node_map))
    for node, links in mapper_graph['links'].items():
        source_id = _node_map[node]
        for link in links:
            target_id = _node_map[link]
            _g.add_edge(_g.vertex(source_id), _g.vertex(target_id))
            
    return _g, _node_map

In [11]:
# This is the callback that will run when we click the button
# The only argument is the button, where we'll save the mapper graph for potential later use

def use_mapper(b):
    _lens = get_lens(lens_selector.value)
    _clusterer = get_clusterer(clusterer_selector.value)
    _cover = km.Cover(n_cubes=cubes_slider.value,
                     perc_overlap=overlap_slider.value)
    _mapper = km.KeplerMapper(verbose=0)
    _graph = _mapper.map(lens=_lens, X=X, cover=_cover, clusterer=_clusterer)
    _g, _ = mapper_to_graph(_graph)
    with out_image:
        clear_output()
        gt.graph_draw(_g, output_size=(600,600));
    b.graph = _graph


run_button.on_click(use_mapper)

In [12]:
display(layout)

## Plot by Variable
Now that you have a map, it can be helpful to view the map by plotting the variation among a specific variable. This takes all the observations that are a part of a node and calculates the mean value for them with any variable.

Remember that a node in the mapper network is the abstract idea of a *pattern* of responses, and row-level observations can appear in multiple nodes (if they can be described with multiple patterns).

⚠️ If you dont have a graph defined above, you'll need to run at least one before proceeding.

### Define "Global" Variables

In [24]:
try:
    graph = run_button.graph
    g, nmap = mapper_to_graph(graph)
    columns = list(X.columns)
except AttributeError:
    display(HTML("<h1 style='color: blue'>⚠️ Please use widgets above to run the mapper algorithm and define a graph</h1>"))

### Define Helper Functions

In [25]:
def assign_position(_g, reset=False):
    if not _g.vertex_properties.get("pos", None) or reset:
        pos = gt.sfdp_layout(_g)
        _g.vertex_properties["pos"] = pos
        
def get_node_means(df, column, mapper_graph, node_map):
    node_means = {}
    for n, i in node_map.items():
        rows = mapper_graph['nodes'][n]
        mean = df.loc[rows, column].mean()
        node_means[i] = mean
    return [v for _, v in sorted(node_means.items())]

### Create Node Color Map Viz

In [26]:
col_dropdown = widgets.Select(
    options=columns, value=columns[0], description="Column", rows=25, disabled=False
)


def draw_column_means(col):
    _vmeans = get_node_means(X, col, graph, nmap)
    _vprop_col = g.new_vertex_property("float", vals=_vmeans)
    assign_position(g)
    gt.graph_draw(
        g,
        pos=g.vp.pos,
        vertex_fill_color=_vprop_col,
        vcmap=(matplotlib.cm.viridis, .5),
        output_size=(600, 600),
    )


out = widgets.interactive_output(draw_column_means, {"col": col_dropdown})
out.layout.height = "600px"
out.layout.width = "600px"

widgets.HBox([out, col_dropdown])

## Categorical Node Coloring
One of the nice things we can do since nodes are circles is draw pie charts over them. We can use this to see if the distribution of categorical variables varies per node!

### Declare "Global" Variables

In [27]:
categorical_columns = ['race', 'engnat', 'gender', 'hand', 'source', 'country']

### Create Helper Functions

In [28]:
def get_node_categorical_distribution(df, column, mapper_graph, node_map):
    node_category_distribution = {}
    category_order = list(df.loc[:, column].value_counts().index)
    for n, i in node_map.items():
        rows = mapper_graph["nodes"][n]
        category_value_counts = (
            df.loc[rows, column].value_counts(normalize=True).to_dict()
        )
        node_category_distribution[i] = [
            category_value_counts.get(c, 0.0) for c in category_order
        ]
    return [v for _, v in sorted(node_category_distribution.items())], category_order


def clamp(x):
    if isinstance(x, float):
        x = int(x * 255)
    return max(0, min(x, 255))


def color_text(text, hex_code):
    return f"<p style='color: {hex_code}'>{text}</p>"


### Create categorical color maps
We're using the Tableau20 color map from matplotlib, but since the order is paired by color (eg dark blue, light blue), we're going to alternate the colors so that we can better cycle through. We'll also convert to hex so that we can create a legend.

In [29]:
tab_20_alt = matplotlib.cm.tab20.colors[0::2] + matplotlib.cm.tab20.colors[1::2]
tab_20_alt_hex = ["#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b)) for (r, g, b) in tab_20_alt]

### Cateogorical Node Color Viz

In [30]:
cat_col_dropdown = widgets.Select(
    options=categorical_columns, value=categorical_columns[0], description="Column", rows=10
)


def draw_cat_column_dist(col):
    _vdist, _  = get_node_categorical_distribution(df, col, graph, nmap)
    _vprop_col = g.new_vertex_property("vector<float>", vals=_vdist)
    assign_position(g)
    gt.graph_draw(
        g,
        pos=g.vp.pos,
        vertex_shape="pie",
        vertex_pie_fractions=_vprop_col,
        vertex_pie_colors=tab_20_alt,
        output_size=(600, 600),
    )

def draw_cat_legend(col):
    _, _cat_order  = get_node_categorical_distribution(df, col, graph, nmap)
    legend_title_html = "<p><strong>Legend</strong></p>"
    legend_color_html = "".join([color_text(cat, color) for cat, color in zip(_cat_order, tab_20_alt_hex)])
    display(HTML(legend_title_html + legend_color_html))


out_cat = widgets.interactive_output(draw_cat_column_dist, {"col": cat_col_dropdown})
out_cat.layout.height = "600px"
out_cat.layout.width = "600px"

out_legend = widgets.interactive_output(draw_cat_legend, {"col": cat_col_dropdown})

widgets.HBox([out_cat, widgets.VBox([cat_col_dropdown, out_legend])])